In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import classification_report, f1_score

# ---------------------------------------------------------
# 1. 데이터 생성 (불균형한 다중 분류 데이터 예시)
# ---------------------------------------------------------
# 1000개의 샘플, 3개의 클래스, 불균형 비율 설정
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10,
                           n_classes=3, weights=[0.1, 0.3, 0.6], random_state=42)

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ---------------------------------------------------------
# 2. 클래스 불균형 처리 (Sample Weight 계산)
# ---------------------------------------------------------
# 다중 분류에서는 scale_pos_weight 대신 각 샘플에 가중치를 부여합니다.
# 희소한 클래스일수록 높은 가중치를 갖게 됩니다.
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

# ---------------------------------------------------------
# 3. 모델 및 GridSearchCV 설정
# ---------------------------------------------------------
# XGBClassifier 초기화
xgb_clf = xgb.XGBClassifier(
    objective='multi:softprob', # 다중 분류 확률
    num_class=3,                # 클래스 개수
    tree_method='hist',         # 속도 및 범주형 변수 처리에 유리
    device='cpu',               # GPU 사용 시 'cuda'
    enable_categorical=True,    # 범주형 데이터 지원 (필요 시)
    eval_metric='mlogloss',
    early_stopping_rounds = 50,# 다중 분류 평가 지표
    random_state=42
)

# 튜닝할 파라미터 그리드
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5], # 결측/노이즈가 많을 때 중요
    'n_estimators': [1000]         # Early Stopping을 위해 충분히 크게 설정
}

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='f1_macro', # 불균형 데이터이므로 Macro F1 사용 권장
    cv=3,
    verbose=1,
    n_jobs=-1
)

# ---------------------------------------------------------
# 4. 학습 실행 (Early Stopping 포함)
# ---------------------------------------------------------
print("GridSearchCV 수행 중...")

# fit 파라미터에 early_stopping 관련 설정 주입
fit_params = {
    'eval_set': [(X_train, y_train), (X_test, y_test)], # 학습 곡선을 위해 Train/Test 모두 전달
    'sample_weight': sample_weights, # [중요] 불균형 처리를 위한 가중치 적용
    'verbose': False
}

grid_search.fit(X_train, y_train, **fit_params)

# 최적의 모델 추출
best_model = grid_search.best_estimator_

print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"최고 CV 점수 (Macro F1): {grid_search.best_score_:.4f}")

# ---------------------------------------------------------
# 5. 최종 평가
# ---------------------------------------------------------
y_pred = best_model.predict(X_test)
print("\n[Classification Report]")
print(classification_report(y_test, y_pred))

# ---------------------------------------------------------
# 6. 학습 곡선 (Learning Curve) 시각화
# ---------------------------------------------------------
results = best_model.evals_result()
epochs = len(results['validation_0']['mlogloss'])
x_axis = range(0, epochs)

fig, ax = plt.subplots(figsize=(10, 6))

# Log Loss 차트
ax.plot(x_axis, results['validation_0']['mlogloss'], label='Train')
ax.plot(x_axis, results['validation_1']['mlogloss'], label='Test (Validation)')
ax.legend()
plt.ylabel('Log Loss')
plt.xlabel('Epochs')
plt.title('XGBoost Learning Curve (Log Loss)')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import classification_report, confusion_matrix

# =============================================================================
# 1. 데이터 생성 (가상의 "지저분한" 데이터 만들기)
# =============================================================================
# 2000개의 샘플, 심각한 불균형 (Class 0: 5%, Class 1: 15%, Class 2: 80%)
X, y = make_classification(n_samples=2000, n_features=15, n_informative=10,
                           n_classes=3, weights=[0.05, 0.15, 0.8], random_state=42)

# DataFrame 변환
feature_names = [f"feature_{i}" for i in range(X.shape[1])]
df_X = pd.DataFrame(X, columns=feature_names)
df_y = pd.Series(y)

# [시나리오 적용] 1. 결측치(NaN) 강제 주입 (전체 데이터의 20%)
for col in df_X.columns:
    df_X.loc[df_X.sample(frac=0.2).index, col] = np.nan

# [시나리오 적용] 2. 범주형 변수 변환 (마지막 3개 컬럼을 범주형으로 가정)
cat_cols = feature_names[-3:]
for col in cat_cols:
    # 임의로 구간을 나누어 범주화 (예: 'A', 'B', 'C')
    df_X[col] = pd.cut(df_X[col], bins=3, labels=['A', 'B', 'C'])
    # ★ 중요: XGBoost Native Support를 위해 'category' 타입으로 변환
    df_X[col] = df_X[col].astype("category")

print(">>> 데이터 준비 완료")
print(f"결측치 개수: {df_X.isna().sum().sum()}")
print(f"데이터 타입 확인:\n{df_X.dtypes.tail(4)}")
print("-" * 50)

# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42, stratify=df_y)

# =============================================================================
# 2. 강력한 클래스 가중치 (Sample Weight) 생성
# =============================================================================
# Class 0(5%)을 놓치지 않기 위해 가중치를 아주 강하게 설정
# 기본 balanced 가중치보다 Class 0에 더 가산점을 줌
custom_class_weights = {
    0: 20.0,  # 소수 클래스 (매우 중요)
    1: 3.0,   # 중간 클래스
    2: 1.0    # 다수 클래스
}

# 설정한 비율대로 각 샘플마다 가중치 부여
train_sample_weights = compute_sample_weight(
    class_weight=custom_class_weights,
    y=y_train
)

print(">>> 클래스별 부여된 가중치 예시:")
for c in [0, 1, 2]:
    idx = np.where(y_train == c)[0][0]
    print(f"Class {c}: {train_sample_weights[idx]:.1f}")
print("-" * 50)

# =============================================================================
# 3. 모델 및 GridSearchCV 설정
# =============================================================================
xgb_clf = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    tree_method='hist',       # [핵심] 결측치/범주형 처리에 필수
    enable_categorical=True,  # [핵심] 범주형 변수 자동 처리
    device='cpu',
    eval_metric='mlogloss',
    early_stopping_rounds = 50,
    seed=42
)

# 결측치가 많고 노이즈가 많은 데이터에 맞춘 파라미터 그리드
param_grid = {
    'n_estimators': [1000],          # Early Stopping용 (충분히 크게)
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],             # 깊이를 제한하여 과적합 방지
    'min_child_weight': [5, 10],     # [핵심] 결측치 노이즈 방지를 위해 높게 설정
    'colsample_bytree': [0.7],       # 변수 샘플링
    'reg_alpha': [1.0, 5.0]          # [핵심] L1 정규화로 불필요한 변수 제거 효과
}

grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='f1_macro',              # 불균형 데이터이므로 Macro F1 사용
    cv=3,
    verbose=1,
    n_jobs=-1
)

# =============================================================================
# 4. 학습 실행 (Early Stopping 포함)
# =============================================================================
# fit_params에 sample_weight와 eval_set을 모두 전달
fit_params = {
    'eval_set': [(X_train, y_train), (X_test, y_test)],
    'sample_weight': train_sample_weights, # ★ 가중치 적용
    'verbose': False
}

print(">>> GridSearchCV 학습 시작...")
grid_search.fit(X_train, y_train, **fit_params)

best_model = grid_search.best_estimator_
print(f"\n>>> 최적 파라미터: {grid_search.best_params_}")
print(f">>> 최고 CV 점수 (Macro F1): {grid_search.best_score_:.4f}")

# =============================================================================
# 5. 최종 평가 및 시각화
# =============================================================================
# 예측
y_pred = best_model.predict(X_test)

# [결과 1] Classification Report
print("\n" + "="*50)
print("Classification Report")
print("="*50)
print(classification_report(y_test, y_pred))

# [결과 2] Confusion Matrix 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Class 0', 'Class 1', 'Class 2'],
            yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# [결과 3] 학습 곡선 (Learning Curve) 시각화
results = best_model.evals_result()
epochs = len(results['validation_0']['mlogloss'])
x_axis = range(0, epochs)

plt.subplot(1, 2, 2)
plt.plot(x_axis, results['validation_0']['mlogloss'], label='Train')
plt.plot(x_axis, results['validation_1']['mlogloss'], label='Test')
plt.legend()
plt.ylabel('Log Loss')
plt.xlabel('Epochs')
plt.title('XGBoost Learning Curve (Log Loss)')
plt.grid(True)

plt.tight_layout()
plt.show()